# Spark with Delta Lake in Jupyter Notebooks

There are several things that need special attention.

- You need the config `"spark.jars.packages", "io.delta:delta-spark_2.13:4.0.0"` in order to download the Delta Lake jvm artifact.
- You need the config `"spark.sql.warehouse.dir", "./spark-warehouse"` to set the location of the Spark SQL warehouse. This will be the location where managed tables are stored.
- You need the config `"javax.jdo.option.ConnectionURL", "jdbc:derby:;databaseName=./metastore/metastore_db;create=true"` and `enableHiveSupport()` in order to use the Hive metastore to manage tables. This will create a local Derby database to store the metadata of the tables and will allow to look up tables by name.
- You need to have the `sparksql-magic` extension installed and `%load_ext sparksql_magic` in order to be able to run SQL queries in a cell magic using `%%sparksql`.
- Alternatively, and maybe it is easier, you can use the DataFrame API and point to the datafiles directly. In that case you don't need the SQL Warehouse, the Hive metastore and the `sparksql-magic` extension.

In [ ]:
%load_ext sparksql_magic

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Row
from pyspark.sql.window import Window
from delta.tables import DeltaTable

spark = (
    SparkSession.builder 
    .appName("DeltaExample")
    .master("local[*]")
    .config("spark.ui.enabled", "true")   
    .config("spark.jars.packages", "io.delta:delta-spark_2.13:4.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.warehouse.dir", "./spark-warehouse")
    .config("javax.jdo.option.ConnectionURL", "jdbc:derby:;databaseName=./metastore/metastore_db;create=true")
    .enableHiveSupport()    # persist metastore across sessions (embedded Hive metastore)
    .getOrCreate()
)

# Show the SparkUI url (useful for monitoring and debuging)
spark.sparkContext.uiWebUrl

In [ ]:
%%sparksql
create database if not exists marvel_db;

In [ ]:
%%sparksql
use marvel_db;

In [ ]:
%%sparksql
describe database marvel_db;

In [ ]:
%%sparksql
show tables;

In [ ]:
%%sparksql
create table if not exists marvel_db.superheroes
(
  id INT,
  hero_name STRING,
  secret_identity STRING,
  power_level INT
)
using delta

In [ ]:
%%sparksql
describe table marvel_db.superheroes

In [ ]:
schema = T.StructType([
    T.StructField("id", T.IntegerType(), False),
    T.StructField("hero_name", T.StringType(), False),
    T.StructField("secret_identity", T.StringType(), False),
    T.StructField("power_level", T.IntegerType(), False)
])

In [ ]:
raw_df = (
    spark.read
        .schema(schema)
        .option("header", "true")
        .csv("data/marvel.csv")
)

raw_df.createOrReplaceTempView("superheroes_raw")

In [ ]:
%%sparksql
select * from superheroes_raw

In [ ]:
%%sparksql
INSERT INTO marvel_db.superheroes (
  id,
  hero_name,
  secret_identity,
  power_level
)
SELECT
  id,
  hero_name,
  secret_identity,
  power_level
FROM superheroes_raw;

In [46]:
%%sparksql
select * from superheroes;

id,hero_name,secret_identity,power_level
1,Iron Man,Tony Stark,97
2,Captain America,Steve Rogers,88
3,Thor,null,98
4,Hulk,Bruce Banner,97
5,Black Widow,Natasha Romanoff,75
6,Spider-Man,Peter Parker,92
7,Black Panther,T'Challa,89
9,Scarlet Witch,Wanda Maximoff,94
10,Hawkeye,Clint Barton,70
11,Ant-Man,Scott Lang,80


In [ ]:
raw_df_mod = (
    spark.read
        .schema(schema)
        .option("header", "true")
        .csv("data/marvel_mod.csv")
)

raw_df_mod.createOrReplaceTempView("superheroes_raw_mod")

In [ ]:
%%sparksql
MERGE INTO superheroes AS target
USING superheroes_raw_mod AS source
  ON target.id = source.id
WHEN MATCHED THEN UPDATE SET
  target.id = source.id,
  target.hero_name = source.hero_name,
  target.secret_identity = source.secret_identity,
  target.power_level = source.power_level
WHEN NOT MATCHED THEN INSERT (
  id, hero_name, secret_identity, power_level
) VALUES (
  source.id, source.hero_name, source.secret_identity, source.power_level
)
WHEN NOT MATCHED BY SOURCE THEN DELETE;

In [48]:
%%sparksql
select * from superheroes;

id,hero_name,secret_identity,power_level
1,Iron Man,Tony Stark,95
2,Captain America,Steve Rogers,88
3,Thor,Thor Odinson,98
4,Hulk,Bruce Banner,97
5,Black Widow,Natasha Romanoff,75
6,Spider-Man,Peter Parker,92
7,Black Panther,T'Challa,89
8,Doctor Strange,Stephen Strange,93
9,Scarlet Witch,Wanda Maximoff,94
10,Hawkeye,Clint Barton,70


In [ ]:
%%sparksql
UPDATE superheroes
SET power_level = power_level + 1
WHERE hero_name = 'Iron Man'

In [ ]:
%%sparksql
DELETE FROM superheroes WHERE hero_name = 'Ant-Man'

In [49]:
%%sparksql
DESCRIBE HISTORY marvel_db.superheroes;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
11,2025-10-17 22:57:04.630000,null,null,RESTORE,"{'version': '1', 'timestamp': None}",null,null,null,10,Serializable,False,"{'removedFilesSize': '1642', 'tableSizeAfterRestore': '1679', 'numRemovedFiles': '1', 'restoredFilesSize': '1679', 'numOfFilesAfterRestore': '1', 'numRestoredFiles': '1'}",null,Apache-Spark/4.0.1 Delta-Lake/4.0.0
10,2025-10-17 22:56:17.177000,null,null,RESTORE,"{'version': '2', 'timestamp': None}",null,null,null,9,Serializable,False,"{'removedFilesSize': '1785', 'tableSizeAfterRestore': '1642', 'numRemovedFiles': '1', 'restoredFilesSize': '1642', 'numOfFilesAfterRestore': '1', 'numRestoredFiles': '1'}",null,Apache-Spark/4.0.1 Delta-Lake/4.0.0
9,2025-10-17 22:22:45.441000,null,null,DELETE,"{'predicate': '[""(hero_name#2530 = Ant-Man)""]'}",null,null,null,8,Serializable,False,"{'numDeletionVectorsUpdated': '0', 'numAddedFiles': '1', 'executionTimeMs': '1817', 'numDeletionVectorsRemoved': '0', 'numRemovedFiles': '1', 'rewriteTimeMs': '260', 'numRemovedBytes': '1816', 'scanTimeMs': '1556', 'numCopiedRows': '18', 'numDeletionVectorsAdded': '0', 'numAddedChangeFiles': '0', 'numDeletedRows': '1', 'numAddedBytes': '1785'}",null,Apache-Spark/4.0.1 Delta-Lake/4.0.0
8,2025-10-17 22:22:41.703000,null,null,UPDATE,"{'predicate': '[""(hero_name#2073 = Iron Man)""]'}",null,null,null,7,Serializable,False,"{'numDeletionVectorsUpdated': '0', 'numAddedFiles': '1', 'executionTimeMs': '1350', 'numDeletionVectorsRemoved': '0', 'numUpdatedRows': '2', 'numRemovedFiles': '1', 'rewriteTimeMs': '311', 'numRemovedBytes': '1817', 'scanTimeMs': '1036', 'numCopiedRows': '17', 'numDeletionVectorsAdded': '0', 'numAddedChangeFiles': '0', 'numAddedBytes': '1816'}",null,Apache-Spark/4.0.1 Delta-Lake/4.0.0
7,2025-10-17 22:22:36.613000,null,null,MERGE,"{'matchedPredicates': '[{""actionType"":""update""}]', 'predicate': '[""(id#984 = id#980)""]', 'notMatchedBySourcePredicates': '[{""actionType"":""delete""}]', 'notMatchedPredicates': '[{""actionType"":""insert""}]'}",null,null,null,6,Serializable,False,"{'numOutputRows': '19', 'numTargetBytesAdded': '1817', 'numTargetRowsInserted': '1', 'numTargetRowsMatchedDeleted': '0', 'numTargetFilesAdded': '1', 'materializeSourceTimeMs': '122', 'numTargetFilesRemoved': '2', 'numTargetRowsMatchedUpdated': '18', 'executionTimeMs': '2457', 'numTargetDeletionVectorsUpdated': '0', 'numTargetRowsCopied': '0', 'rewriteTimeMs': '685', 'numTargetRowsUpdated': '18', 'numTargetDeletionVectorsRemoved': '0', 'numTargetRowsDeleted': '1', 'scanTimeMs': '1637', 'numSourceRows': '10', 'numTargetDeletionVectorsAdded': '0', 'numTargetChangeFilesAdded': '0', 'numTargetRowsNotMatchedBySourceUpdated': '0', 'numTargetRowsNotMatchedBySourceDeleted': '1', 'numTargetBytesRemoved': '3300'}",null,Apache-Spark/4.0.1 Delta-Lake/4.0.0
6,2025-10-17 22:22:24.883000,null,null,WRITE,"{'mode': 'Append', 'partitionBy': '[]'}",null,null,null,5,Serializable,True,"{'numOutputRows': '10', 'numOutputBytes': '1679', 'numFiles': '1'}",null,Apache-Spark/4.0.1 Delta-Lake/4.0.0
5,2025-10-17 22:17:58.624000,null,null,UPDATE,"{'predicate': '[""(hero_name#5892 = Iron Man)""]'}",null,null,null,4,Serializable,False,"{'numDeletionVectorsUpdated': '0', 'numAddedFiles': '1', 'executionTimeMs': '857', 'numDeletionVectorsRemoved': '0', 'numUpdatedRows': '1', 'numRemovedFiles': '1', 'rewriteTimeMs': '271', 'numRemovedBytes': '1620', 'scanTimeMs': '585', 'numCopiedRows': '8', 'numDeletionVectorsAdded': '0', 'numAddedChangeFiles': '0', 'numAddedBytes': '1621'}",null,Apache-Spark/4.0.1 Delta-Lake/4.0.0
4,2025-10-17 22:16:59.135000,null,null,DELETE,"{'predicate': '[""(hero_name#2617 = Ant-Man)""]'}",null,null,null,3,Serializable,False,"{'numDeletionVectorsUpdated': '0', 'numAddedFiles': '1', 'executionTimeMs': '2124', 'numDeletionVectorsRemoved': '0', 'numRemovedFiles': '1', 'rewriteTimeMs': '324', 'n

In [41]:
%%sparksql
DESCRIBE DETAIL marvel_db.superheroes;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures
delta,e9b18029-c184-4b57-b261-32dcb2e54d5e,spark_catalog.marvel_db.superheroes,null,file:/home/yannis/Development/tmp/pyspark-delta/spark-warehouse/marvel_db.db/superheroes,2025-10-17 22:16:33.753000,2025-10-17 22:22:45.441000,[],[],1,1785,{},1,2,"['appendOnly', 'invariants']"


In [ ]:
%%sparksql
SELECT * FROM marvel_db.superheroes VERSION AS OF 1;

In [ ]:
%%sparksql
SELECT * FROM marvel_db.superheroes VERSION AS OF 2;

In [43]:
%%sparksql
SELECT * FROM marvel_db.superheroes VERSION AS OF 9;

id,hero_name,secret_identity,power_level
1,Iron Man,Tony Stark,98
1,Iron Man,Tony Stark,98
2,Captain America,Steve Rogers,88
2,Captain America,Steve Rogers,88
3,Thor,null,98
3,Thor,null,98
4,Hulk,Bruce Banner,97
4,Hulk,Bruce Banner,97
5,Black Widow,Natasha Romanoff,75
5,Black Widow,Natasha Romanoff,75


In [ ]:
# %%sparksql
# SELECT * FROM marvel_db.superheroes TIMESTAMP AS OF '2025-10-17 17:56:00';

In [ ]:
# %%sparksql
# SELECT * FROM marvel_db.superheroes TIMESTAMP AS OF '2025-10-17 17:57:00';

In [ ]:
# %%sparksql
# SELECT * FROM marvel_db.superheroes TIMESTAMP AS OF '2025-10-17 17:57:07.292000';

In [54]:
%%sparksql
RESTORE TABLE marvel_db.superheroes TO VERSION AS OF 1;

25/10/17 23:00:28 WARN DAGScheduler: Broadcasting large task binary with size 1089.8 KiB
                                                                                

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
1679,1,0,0,0,0


In [57]:
%%sparksql
RESTORE TABLE marvel_db.superheroes TO TIMESTAMP AS OF '2025-10-17 22:35:00';

25/10/17 23:01:42 WARN DAGScheduler: Broadcasting large task binary with size 1089.8 KiB
                                                                                

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
1785,1,1,1,1679,1785
